In [1]:
def calculate_dc(klines, period=20):
    """
    计算 Donchian Channel (DC)
    :param klines: 数据帧，包含 'High' 和 'Low' 列
    :param period: 周期（默认20）
    :return: 包含DC Upper, DC Lower 和 DC Middle的 DataFrame
    """
    klines['DC_Upper'] = klines['High'].rolling(window=period).max()
    klines['DC_Lower'] = klines['Low'].rolling(window=period).min()
    klines['DC_Middle'] = (klines['DC_Upper'] + klines['DC_Lower']) / 2
    return klines[['DC_Upper', 'DC_Lower', 'DC_Middle']]

In [2]:
def calculate_kc(klines, period=20, multiplier=1):
    """
    计算 Keltner Channel (KC)
    :param klines: 数据帧，包含 'Close', 'High', 'Low' 列
    :param period: 周期（默认20）
    :param multiplier: ATR的乘数（默认1）
    :return: 包含KC Upper, KC Lower 和 KC Middle的 DataFrame
    """
    klines['EMA'] = klines['Close'].ewm(span=period, adjust=False).mean()
    klines['ATR'] = klines['High'].combine(klines['Low'], max) - klines['Close'].combine(klines['Low'], min)
    klines['ATR'] = klines['ATR'].rolling(window=period).mean()
    
    klines['KC_Upper'] = klines['EMA'] + multiplier * klines['ATR']
    klines['KC_Lower'] = klines['EMA'] - multiplier * klines['ATR']
    klines['KC_Middle'] = klines['EMA']
    return klines[['KC_Upper', 'KC_Lower', 'KC_Middle']]

In [3]:
def calculate_ichimoku(klines, conversion_line_period=9, base_line_period=26, leading_span_b_period=52):
    """
    计算 Ichimoku Cloud (一目均衡图)
    :param klines: 数据帧，包含 'High' 和 'Low' 列
    :param conversion_line_period: 转换线周期（默认9）
    :param base_line_period: 基准线周期（默认26）
    :param leading_span_b_period: 领先跨度B周期（默认52）
    :return: 包含所有Ichimoku指标的 DataFrame
    """
    klines['Tenkan_Sen'] = (klines['High'].rolling(window=conversion_line_period).max() + klines['Low'].rolling(window=conversion_line_period).min()) / 2
    klines['Kijun_Sen'] = (klines['High'].rolling(window=base_line_period).max() + klines['Low'].rolling(window=base_line_period).min()) / 2
    klines['Senkou_Span_A'] = (klines['Tenkan_Sen'] + klines['Kijun_Sen']) / 2
    klines['Senkou_Span_B'] = (klines['High'].rolling(window=leading_span_b_period).max() + klines['Low'].rolling(window=leading_span_b_period).min()) / 2
    klines['Chikou_Span'] = klines['Close'].shift(-base_line_period)
    
    return klines[['Tenkan_Sen', 'Kijun_Sen', 'Senkou_Span_A', 'Senkou_Span_B', 'Chikou_Span']]

In [4]:
def calculate_ema(klines, period=13):
    """
    计算 Exponential Moving Average (EMA)
    :param klines: 数据帧，包含 'Close' 列
    :param period: 周期（默认13）
    :return: 包含EMA的 DataFrame
    """
    klines[f'EMA_{period}'] = klines['Close'].ewm(span=period, adjust=False).mean()
    return klines[[f'EMA_{period}']]

In [5]:
def calculate_ma(klines, period=7):
    """
    计算 Simple Moving Average (MA)
    :param klines: 数据帧，包含 'Close' 列
    :param period: 周期（默认7）
    :return: 包含MA的 DataFrame
    """
    klines[f'MA_{period}'] = klines['Close'].rolling(window=period).mean()
    return klines[[f'MA_{period}']]

In [6]:
def calculate_kdj(klines, k_period=14, d_period=3, j_period=3):
    """
    计算 Stochastic Oscillator (KDJ)
    :param klines: 数据帧，包含 'Close', 'High', 'Low' 列
    :param k_period: %K的周期（默认14）
    :param d_period: %D的周期（默认3）
    :param j_period: %J的周期（默认3）
    :return: 包含K, D, J的 DataFrame
    """
    low_min = klines['Low'].rolling(window=k_period).min()
    high_max = klines['High'].rolling(window=k_period).max()
    klines['K'] = 100 * (klines['Close'] - low_min) / (high_max - low_min)
    klines['D'] = klines['K'].rolling(window=d_period).mean()
    klines['J'] = 3 * klines['K'] - 2 * klines['D']
    return klines[['K', 'D', 'J']]

In [7]:
def calculate_macd(klines, fast_period=12, slow_period=26, signal_period=9):
    """
    计算 Moving Average Convergence Divergence (MACD)
    :param klines: 数据帧，包含 'Close' 列
    :param fast_period: 快速EMA周期（默认12）
    :param slow_period: 慢速EMA周期（默认26）
    :param signal_period: 信号线周期（默认9）
    :return: 包含MACD, Signal, Histogram的 DataFrame
    """
    klines['EMA_Fast'] = klines['Close'].ewm(span=fast_period, adjust=False).mean()
    klines['EMA_Slow'] = klines['Close'].ewm(span=slow_period, adjust=False).mean()
    klines['MACD'] = klines['EMA_Fast'] - klines['EMA_Slow']
    klines['Signal'] = klines['MACD'].ewm(span=signal_period, adjust=False).mean()
    klines['Histogram'] = klines['MACD'] - klines['Signal']
    return klines[['MACD', 'Signal', 'Histogram']]

In [8]:
import pandas as pd

# 读取加密货币历史数据 (以2小时级别为例)
file_path = 'BTCUSDT+2h.csv'  # 你的数据文件路径
klines = pd.read_csv(file_path, parse_dates=['time'], index_col='time')

# 计算各个指标
klines_dc = calculate_dc(klines, period=120)  # DC
klines_kc = calculate_kc(klines, period=20, multiplier=1)  # KC
klines_ich = calculate_ichimoku(klines, conversion_line_period=9, base_line_period=26, leading_span_b_period=52)  # Ichimoku
klines_ema = calculate_ema(klines, period=13)  # EMA
klines_ma = calculate_ma(klines, period=7)  # MA
klines_kdj = calculate_kdj(klines, k_period=30, d_period=30, j_period=30)  # KDJ
klines_macd = calculate_macd(klines, fast_period=20, slow_period=60, signal_period=16)  # MACD

# 合并所有指标数据
klines = pd.concat([klines, klines_dc, klines_kc, klines_ich, klines_ema, klines_ma, klines_kdj, klines_macd], axis=1)

# 导出数据到CSV文件
output_file_path = 'BTCUSDT_with_indicators_2h.csv'  # 设定文件路径
klines.to_csv(output_file_path)

print(f"Data with indicators exported to {output_file_path}")
klines

FileNotFoundError: [Errno 2] No such file or directory: 'BTCUSDT+2h.csv'